### Dense word embeddings { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_10.pdf) }
* Easier to use as features
* Generalize better 
* Capture synonymy better
* **word2vec**:
    * Train classifier on binary prediction task - is $w$ likely to show up near "apricot"?
    * Implicitly learns embeddings
    * Skip-gram algorithm
        * Target + neighbor = positive samples, random negative samples
        * Logistic regression classifier - weights are embeddings 
        * Model for a pair of words $t,c$:  
            $ P(+|t,c) = \frac{1}{1+e^{-t \cdot c}} $  
            $ P(-|t,c) = 1 - P(+|t,c) $  
        * Skip-gram for context words  
        $P(+|t,c_{1:k}) = \prod_{i=1}^k \frac{1}{1+e^{-t \cdot c_i}} $  
        $log P(+|t,c_{1:k}) = \sum{i=1}^k log \frac{1}{1+e^{-t \cdot c_i}} $  
        * Training:
            * Assume a context window
            * For each positive example, create $k$ negative random examples (noise words)
            * Can pick noise word $w$ according to $P_\alpha(w) = \frac{count(w)^\alpha}{\sum_w count(w)^\alpha}$, $\alpha = .75$
            * Maximize $\sum_{(t,c)\in +} log P(+|t,c) + \sum_{(t,c)\in -}log P(-|t,c) $ (max prob of neg pairs being labeled as neg, max prob of pos pairs being labeled as pos)
            * Throw away classifier & keep embeddings
* Limitations:
    * Multiple senses not captured
    * Cosine similarity doesn't help with antonyms vs synonyms
    * Reflect bias
       

### Machine translation { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_11.pdf) }
* History
    * Rule-based --> statistical --> neural
    * Right now, MT good for: 
        * Understanding content
        * Communication
        * Making content available in other languages
* Challenges: Ambiguity, word order
* Evaluation
    * Adequacy: Conservation of meaning
    * Fluency
    * BLEU:
        * $n$-grams of size 1 to 4
        * $min(1, \frac{out\_len}{ref\_len})(\prod_{i=1}^4 precision_i)^{\frac{1}{4}}$
        * Computed over entire corpus
        * Drawbacks: All words equally weighted, scores meaningless
        

### RNNs { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_12.pdf) }
* Can capture LD dependencies
* Params:
    * $W_{mh}: |H| * |V|$ ($V$ = embedding size)
    * $W_{hh}: |H| * |H|$ 
    * $W_{hs}: |V| * |H|$
* Update eqns:
    * $m_t = M_{.,e_{t-1}} $
    * $h_t = tanh(W_{mh}m_t + W_{hh}h_{t-1} + b_n) $ aka $RNN(m_t, h_{t-1})$
    * $p_t = softmax(W_{hs}h_t + b_s)$
* Training
    * Online: Update weights for each training example
    * Batch: Accumulate gradient over all examples & update at the end




### Neural machine translation { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_13.pdf) }
* $\hat{E} = argmax_E P(E|F;\theta)$
* Encoder-decoder model
    * $m_t^{(f)} = M_{.,f_t}^{(f)}$
    * $h_t^{(f)} = RNN^{(f)}(m_t^{(f)}, h_{t-1}^{(f)})$
    * $m_t^{(e)} = M_{.,e_{t-1}}^{(e)}$
    * $h_t^{(e)} = RNN^{(e)}(m_t^{(e)}, h_{t-1}^{(e)})$
    * $p_t^{(e)} = softmax(W_{hs}h_t^{(e)} + b_s)$
* Search:
    * Ancestral sampling: Randomly generate words one by one
    * Greedy search: Pick highest prob. word at each time step
        * Often generates easy words first
        * Prefers multiple common words to rare words
    * Beam search: Consider b total top hypotheses at each time step (instead of single top)
* Bidirectional encoder
    * $\overrightarrow{h}_t^{(f)} = \overrightarrow{RNN}^{(f)}(m_t^{(f)}, \overrightarrow{h}_{t-1}^{(f)})$
    * $\overleftarrow{h}_t^{(f)} = \overleftarrow{RNN}^{(f)}(m_t^{(f)}, \overleftarrow{h}_{t+1}^{(f)})$
    * $h_0^{(e)} = tanh(W_{\overrightarrow{f}e}\overrightarrow{h}_{|F|} + W_{\overleftarrow{f}e}\overleftarrow{h}_{1} + b_e)$ (Add forward & backward hidden to match input dim)
* Attention mechanism { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_14.pdf) }
    * Decoding - use attention weights to perform linear combination of source sentence word vectors
    * Let $H^{(f)} $ be the concat of all encoder outputs $ [\overleftarrow{h}_j^{(f)}; \overrightarrow{h}_j^{(f)}]$
    * Let context vector $c_t = H^{(f)}\alpha_t$
    * $\alpha_t$: Weights between 0 and 1
    * $h_t^{(e)} = \textrm{enc}([\textrm{embed}(e_{t-1}); c_{t-1}], h_{t-1}^{(e)})$
    * $a_{t,j} = \textrm{attn_score}(h_j^{(f)}, h_t^{(e)}) $
        * Dot product: $h_j^{(f)} \cdot h_t^{(e)}$
            * No extra params, hidden states must be same size
        * Bilinear: $h_j^{(f)} W_a h_t^{(e)}$
            * More params, but dims can be different
        * Multi-layer perceptron (Bahdanau): $w_{a2}^{\intercal}\textrm{tanh}(W_{a1}[h_t^{(e)};  h_j^{(f)}])$
            * Dims can be diff, less new params
    * $\alpha_t = softmax(a_t) $
    * $p_t^{(e)} = softmax(W_{hs}[h_t^{(e)}; c_t] + b_s)$
    * Can help interpret/illustrate translation decisions
    * Help insert translations for OOV words
* Addressing length bias (default models gen short sentences)
    * Prior prob. on sentence length: $\hat{E} = argmax_E log P(|E||F|) + log P(E|F)$
    * Normalize by sentence length:  $\hat{E} = argmax_E log \frac{P(E|F)}{|E|}$
* Works well only with high resources


### Subwords, multi-task, multilingual, semi-supervised ?

### POS tagging & sequence labeling { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_19.pdf) }
* Parts of speech to know:
    * Noun, verb, adjective
    * Adverb: Modify verbs/adjectives
    * Prepositions: Spatial/temporal relations (*on* the shelf) - used with noun phrases
    * Particles: Like prepositions, but with verbs (find *out*, turn *over*)
    * Determiners: Articles (a, an, the), that, this, many, such
    * Pronouns: Refer to entities/persons
    * Conjunctions: 
        * Coordinating: Equal status (and, or)
        * Subordinating: Unequal status (after, while, that)
* POS tagging
    * Uses Penn Treebank tagset
    * Difficult due to ambiguity
    * Perceptron: sequence of tokens x ==> sequence of tags y
        * $ w \leftarrow w + \phi(x,y) - \phi(x,\hat{y}) $
        * $ \hat{y} = \textrm{argmax}_{\hat{y} \in Y(x)} w \cdot \phi(x,\hat{y})$
    * Features (constant wrt input length):
        * Unary: Between $x$ and a single label in $y$ ( # of times word $w$ has been labeled with tag $l$ for all words and tags )
        * Markov: Between adjacent labels in $y$ ( # times tag $l$ adjacent to $l'$ in output for tag pairs $l, l'$
    * Viterbi algorithm for argmax { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_20.pdf) }: 
        * $l^{th}$ word, label $k$: $\alpha_{l,k} = max_{\hat{y}_{1:l-1}} w \cdot \theta_{1:l}(x, \hat{y} \circ k)$
        * Output space becomes $lk^2$
        * Features must *decompose over input* - can't look at predictions ahead or two timestamps back
    * Hamming loss - more nuanced than 0-1 loss
        * Minimize $l^{(Ham)}(y, \hat{y}) = \sum_{l=1}^L 1\{y_l \neq \hat{y}_l\}$
* Information extraction - detecting named entities (BIO labeling scheme: B-PER, I-PER, O, B-ORG, I-ORG, etc)
* Integer Linear Programming { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_21.pdf) }
    * Optimization of the form $max_z a \cdot z $ subj to linear constraints on $z$
    * Markov feature labeling as ILP
        * $z_{l, k', k} = 1\{l = k$ & $ l-1 = k'\}$
        * $a_{l,k',k} = w \cdot \theta_l(x,(...,k',k))$
        * $z_{l,k',k} \in \{0,1\} $
        * For a given $l$, there is one active $z$
        * $z$'s are internally consistent

### Syntax, grammar & parsing { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_21.pdf) }
* Dependency trees: Connected, acyclic, single-head
* Can help NLP tasks by resolving ambiguity & generalizing (LD dependencies)
* Transition-based dependency parsing { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_22.pdf) }
    * Config: stack, buffer, set of arcs
    * Arc standard parsing:
        * SHIFT: Remove word from buffer, push on stack
        * LA: 1st word on stack ==> 2nd, remove 2nd
        * RA: 2nd word on stack ==> 1st, remove 1st
        * Reqs:
            * ROOT cannot have incoming arcs
            * LA can't be applied when ROOT is 2nd element
            * LA, RA reqire 2 elements in stack
        * Weakness: Right dependents cannot be attached to head till all their dependents have been attached
    * Training features for oracle
         * Config + action taken
         * Word forms, POS
    * Arc eager parsing { [x](http://www.cs.umd.edu/class/fall2019/cmsc470/slides/slides_23.pdf) }
         * SHIFT: Remove word from buffer, push on stack
         * RA: Word on stack ==> word on buffer, move buffer head to stack
         * LA: Word on buffer ==> word on stack, pop stack
    * Properties:
        * Soundness: Projective (no crossing arcs)
            * Projective arc: If there is a path from head ==> every word between head and dependent
        * Complete: There is a translation sequene that can generate a projective dep. forest
        * Deal with non projectivity:
            * Transform non-projective tree into projective (so it's reversible)
            * Add new transitions
* Dependency forest: single head, acyclic, not connected
* Graph-based dependency parsing:
    * Find best directed spanning tree of multi-digraph that captures all poss. dependencies
    * Arc-factored model: Weight of graph can be factored as sum/product of weights of arcs
    * Chu-Liu-Edmonds algorithm:
        * Find highest scoring incoming arc for each vertex
            * If this is a tree, we've found MST
            * Else:
                * Identify cycle and **contract**:
                    * Recalculate arc weights into & out of cycle
                    * Outgoing arcs = max of outgoing arc over all vertices in cycle
                    * Incoming arc weights = weight of best spanning tree that includes head of incoming arc & all nodes in cycle